In [ ]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=9ce3fefed2e83bd8ac1b227f82f96ac54784885751e1570560ab8dea549ed312
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install shap


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 7.4 MB/s eta 0:00:00


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False


In [ ]:
import numpy as np
import pandas as pd
import urllib.request
import re
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Dense, Bidirectional, Input, Lambda, GlobalMaxPool1D, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import lime
import lime.lime_text
import shap

# 한국어 불용어 리스트 정의
korean_stopwords = ['은', '는', '이', '가', '을', '를', '의', '에', '에서', '와', '과', '도', '로', '으로', '그리고', '하지만', '그러나', '따라서', '그러므로', '그래서', '또한', '매우', '아주', '더', '가장', '너무', '항상', '자주', '때로', '가끔', '이', '그', '저', '나', '너', '우리', '당신', '여러분']

# 불용어 제거 함수 정의
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in korean_stopwords]
    return ' '.join(filtered_words)

# 데이터 전처리 함수 정의
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', text)

# 특수 단어와 긴급성 표현 존재 여부를 특징으로 추가하는 함수
keywords = ['송금', '입금', '결제', '환불', '수수료', '잔액']
urgent_keywords = ['긴급', '즉시', '필요', '중요', '긴급한', '바로']

def extract_keyword_features(text):
    features = [1 if keyword in text else 0 for keyword in keywords]
    urgency_feature = 1 if any(keyword in text for keyword in urgent_keywords) else 0
    features.append(urgency_feature)
    return np.array(features)

# 데이터 전처리 및 특징 추출 적용
def preprocess_and_extract_features(data):
    data['cleaned_text'] = data['v2'].apply(remove_non_korean).apply(remove_stopwords)
    data['keyword_features'] = data['cleaned_text'].apply(extract_keyword_features)
    return data

# BERT 토크나이저 및 모델 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased', output_attentions=True)

# 데이터 불러오기 및 중복 제거
try:
    urllib.request.urlretrieve("https://raw.githubusercontent.com/tonykorea99/Spam-alart/main/OriginData4.csv", filename='OriginData4.csv')
    data = pd.read_csv('OriginData4.csv', encoding='utf-8')
    data = data.sample(frac=1, random_state=42)
    print('총 샘플 수:', len(data))
except Exception as e:
    print('에러 발생:', e)

data = data[['v1', 'v2']]
data['v1'] = data['v1'].replace(['ham', 'spam'], [0, 1])
data.drop_duplicates(subset=['v2'], inplace=True)
print('중복 제거 후의 전체 샘플 수:', len(data))

# 데이터 전처리 적용
data = preprocess_and_extract_features(data)

# BERT 입력 형식으로 변환
max_length = 100  # 최대 시퀀스 길이
input_ids_list = []
attention_masks_list = []
keyword_features_list = []

for sent, keyword_feats in zip(data['cleaned_text'], data['keyword_features']):
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens=True,
                        max_length=max_length,
                        padding='max_length',
                        truncation=True,
                        return_attention_mask=True,
                        return_tensors='tf',
                   )

    input_ids_list.append(encoded_dict['input_ids'])
    attention_masks_list.append(encoded_dict['attention_mask'])
    keyword_features_list.append(keyword_feats)

input_ids = tf.concat(input_ids_list, axis=0)
attention_masks = tf.concat(attention_masks_list, axis=0)
keyword_features = np.stack(keyword_features_list)

# 레이블 데이터
labels = data['v1'].values

# 데이터 분리
X_train, X_test, y_train, y_test, mask_train, mask_test, feats_train, feats_test = train_test_split(
    input_ids.numpy(), labels, attention_masks.numpy(), keyword_features, test_size=0.2, random_state=42)

# 학습 데이터에서 다시 검증 데이터 분리
X_train, X_val, y_train, y_val, mask_train, mask_val, feats_train, feats_val = train_test_split(
    X_train, y_train, mask_train, feats_train, test_size=0.2, random_state=25)

# 입력 레이어 정의
input_ids_layer = Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
attention_mask_layer = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")
keyword_features_layer = Input(shape=(len(keywords) + 1,), dtype=tf.float32, name="keyword_features")

# BERT 출력
bert_inputs = {'input_ids': input_ids_layer, 'attention_mask': attention_mask_layer}
bert_output = Lambda(lambda x: bert_model(x, output_attentions=True))(bert_inputs)
sequence_output, attentions = bert_output[0], bert_output[-1]

# GRU 레이어
gru_output = Bidirectional(GRU(128, return_sequences=True))(sequence_output)
pooled_output = GlobalMaxPool1D()(gru_output)

# 추가 특징 결합
combined_features = Concatenate()([pooled_output, keyword_features_layer])

# 드롭아웃 및 출력 레이어
dropout = Dropout(0.3)(combined_features)
dense_output = Dense(64, activation='relu')(dropout)
output = Dense(1, activation='sigmoid')(dense_output)

# 모델 정의
model = Model(inputs=[input_ids_layer, attention_mask_layer, keyword_features_layer], outputs=output)

# 모델 컴파일 및 학습
optimizer = Adam(learning_rate=2e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
history = model.fit(
    x={'input_ids': X_train, 'attention_mask': mask_train, 'keyword_features': feats_train},
    y=y_train,
    epochs=5,
    batch_size=64,
    validation_data=({'input_ids': X_val, 'attention_mask': mask_val, 'keyword_features': feats_val}, y_val)
)

# 모델 평가
y_pred = model.predict({'input_ids': X_test, 'attention_mask': mask_test, 'keyword_features': feats_test})
y_pred = (y_pred > 0.5).astype(int)

# 정확도 및 다양한 평가 지표 출력
loss, accuracy = model.evaluate({'input_ids': X_test, 'attention_mask': mask_test, 'keyword_features': feats_test}, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# 다양한 평가 지표 출력
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

총 샘플 수: 91000
중복 제거 후의 전체 샘플 수: 87642


The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'tf_bert_model/bert/embeddings/word_embeddings/weight:0' shape=(119547, 768) dtype=float32>
  <tf.Variable 'tf_bert_model/bert/embeddings/token_type_embeddings/embeddings:0' shape=(2, 768) dtype=float32>
  <tf.Variable 'tf_bert_model/bert/embeddings/position_embeddings/embeddings:0' shape=(512, 768) dtype=float32>
  <tf.Variable 'tf_bert_model/bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32>
  <tf.Variable 'tf_bert_model/bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32>
  <tf.Variable 'tf_bert_model/bert/encoder/layer_._0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>
  <tf.Variable 'tf_bert_model/bert/encoder/layer_._0/attention/self/query/bias:0' shape=(768,) dtype=float32>
  <tf.Variable 'tf_bert_model/bert/encoder/layer_._0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>
  <tf.Variable 'tf_bert_model/bert

Epoch 1/5
877/877 [==============================] - 771s 840ms/step - loss: 0.1023 - accuracy: 0.9667 - val_loss: 0.0184 - val_accuracy: 0.9959
Epoch 2/5
877/877 [==============================] - 773s 881ms/step - loss: 0.0145 - accuracy: 0.9966 - val_loss: 0.0074 - val_accuracy: 0.9980
Epoch 3/5
877/877 [==============================] - 778s 886ms/step - loss: 0.0071 - accuracy: 0.9982 - val_loss: 0.0063 - val_accuracy: 0.9982
Epoch 4/5
877/877 [==============================] - 728s 830ms/step - loss: 0.0048 - accuracy: 0.9986 - val_loss: 0.0056 - val_accuracy: 0.9984
Epoch 5/5
548/548 [==============================] - 136s 247ms/step - loss: 0.0048 - accuracy: 0.9989
Loss: 0.004830059129744768, Accuracy: 0.9988590478897095
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15066
           1       1.00      1.00      1.00      2463

    accuracy                           1.00     17529
   macro avg       1.00      1.00      1.0

In [ ]:
import joblib
# 모델 저장
model.save('/content/drive/MyDrive/spam_detection_model2.h5')
print("모델이 저장되었습니다.")

# 토크나이저 저장
tokenizer.save_pretrained('/content/drive/MyDrive/tokenizer')
print("토크나이저가 저장되었습니다.")

# 전처리 함수 저장
joblib.dump(remove_non_korean, '/content/drive/MyDrive/remove_non_korean_feat.pkl')
print("전처리 함수가 저장되었습니다.")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


모델이 저장되었습니다.
토크나이저가 저장되었습니다.
전처리 함수가 저장되었습니다.


In [ ]:
# 특징 중요도 분석
def get_feature_importance(model, feature_names):
    weights = model.layers[-2].get_weights()[0].flatten()
    feature_importance = {name: weight for name, weight in zip(feature_names, weights)}
    return feature_importance

feature_names = keywords + ['urgency']
feature_importance = get_feature_importance(model, feature_names)
print("Feature Importance:")
for feature, importance in feature_importance.items():
    print(f"{feature}: {importance:.4f}")

# 어텐션 시각화 함수 정의
def visualize_attention(tokens, attentions, attention_mask, layer=0, head=0):
    attention = attentions[layer][0, head].numpy()

    # 패딩 토큰의 행과 열을 제거
    mask = attention_mask.numpy().reshape(-1)
    valid_indices = np.where(mask == 1)[0]
    attention = attention[np.ix_(valid_indices, valid_indices)]
    tokens = [tokens[i] for i in valid_indices]

    plt.figure(figsize=(10, 8))
    sns.heatmap(attention, xticklabels=tokens, yticklabels=tokens, cmap='viridis')
    plt.title(f'Layer {layer + 1}, Head {head + 1} Attention')
    plt.xlabel('Tokens')
    plt.ylabel('Tokens')
    plt.show()

# LIME을 사용한 텍스트 예측 설명
def lime_explanation(model, tokenizer, text):
    explainer = lime.lime_text.LimeTextExplainer(class_names=['Ham', 'Spam'])

    def predict_proba(texts):
        encodings = tokenizer.batch_encode_plus(texts, max_length=100, padding=True, truncation=True, return_tensors='tf')
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']
        features = [extract_keyword_features(text) for text in texts]
        features = np.stack(features)
        probs = model.predict([input_ids, attention_mask, features])
        return np.concatenate([1 - probs, probs], axis=1)

    exp = explainer.explain_instance(text, predict_proba, num_features=10)
    exp.show_in_notebook()

# SHAP을 사용한 텍스트 예측 설명
def shap_explanation(model, tokenizer, text):
    def predict_proba(texts):
        encodings = tokenizer.batch_encode_plus(texts, max_length=100, padding=True, truncation=True, return_tensors='tf')
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']
        features = [extract_keyword_features(text) for text in texts]
        features = np.stack(features)
        probs = model.predict([input_ids, attention_mask, features])
        return np.concatenate([1 - probs, probs], axis=1)

    explainer = shap.Explainer(predict_proba, masker=shap.maskers.Text(tokenizer))
    shap_values = explainer([text])
    shap.plots.text(shap_values[0])

# 사용자로부터 입력받은 텍스트에 대해 스미싱 여부 판단 및 어텐션 시각화
def classify_and_visualize(input_text):
    # 사용자 입력 텍스트 전처리: 한글만 남기고 나머지는 제거
    input_text = remove_non_korean(input_text)
    input_text = remove_stopwords(input_text)  # 불용어 제거 추가

    # BERT 입력 형식으로 변환
    encoded_dict = tokenizer.encode_plus(
                        input_text,
                        add_special_tokens=True,
                        max_length=100,
                        padding='max_length',
                        truncation=True,
                        return_attention_mask=True,
                        return_tensors='tf',
                   )

    # 문장을 BERT 입력으로 변환
    input_ids = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    keyword_features = extract_keyword_features(input_text).reshape(1, -1)

    # 모델 예측
    smishing_score = model.predict([input_ids, attention_mask, keyword_features])[0][0]

    # 스미싱 여부 판단 결과 출력
    if smishing_score > 0.5:
        print("{:.2f}% 확률로 스미싱입니다.".format(smishing_score * 100))
    else:
        print("{:.2f}% 확률로 일반 메시지입니다.".format((1 - smishing_score) * 100))

    # 어텐션 시각화
    bert_output = bert_model(input_ids, attention_mask=attention_mask, output_attentions=True)
    sequence_output, attentions = bert_output[0], bert_output[-1]
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    visualize_attention(tokens, attentions, attention_mask, layer=0, head=0)

    # LIME 설명
    lime_explanation(model, tokenizer, input_text)

    # SHAP 설명
    shap_explanation(model, tokenizer, input_text)

# 사용자로부터 텍스트 입력 받기
user_input_text = input("텍스트를 입력하세요: ")

# 스미싱 여부 판단 및 어텐션 시각화 수행
classify_and_visualize(user_input_text)